# Procesamiento de Audio

Este notebook lleva a cabo varios pasos de procesamiento en señales de audio, incluyendo la reducción de ruido, eliminación de silencios, normalización, y segmentación.

A lo largo del proceso, se generan varios archivos de audio que ilustran cada paso. Al final, se presentan visualizaciones para comparar las señales antes y después del procesamiento.

## Índice
1. [Importación de Bibliotecas](#importación)
2. [Definición de Funciones para el Procesamiento de Audio](#definición)
3. [Proceso de Reducción de Ruido](#proceso_ruido)
4. [Proceso de Eliminación de Silencios](#proceso_silencios)
5. [Proceso de Normalización del Audio](#proceso_normalización)
6. [Proceso de Preenfasis, Segmentación y Ventaneo](#proceso_segmentación)
7. [Archivos generados](#archivos)
8. [Gráficas](#gráficas)

<a id = "importación"></a>
## 1. Importación de Bibliotecas
En esta sección, se importan todas las librerías necesarias para el análisis y procesamiento de audio.

In [ ]:
import numpy as np  # Biblioteca para operaciones matemáticas avanzadas y manejo de arreglos multidimensionales.

# noisereduce es una biblioteca para reducir el ruido en señales de audio.
import noisereduce as nr

# pydub es una biblioteca que proporciona herramientas simples para manipular audio.
from pydub import AudioSegment  # Utilizado para trabajar con segmentos de audio (carga, exporta, operaciones básicas).
from pydub.silence import split_on_silence  # Función para dividir un audio basado en silencios detectados.

# Herramientas para el análisis de señales.
from scipy.fft import fft, fftfreq  # Funciones para realizar transformada rápida de Fourier (FFT) y calcular las frecuencias asociadas.

# librosa es una biblioteca para el análisis y procesamiento de audio.
import librosa  # Biblioteca principal.
import librosa.display  # Herramientas para visualizar datos de audio, como espectrogramas.

# soundfile es utilizado para leer y escribir archivos de audio en diversos formatos.
import soundfile as sf  

# Bibliotecas estándar de Python para trabajar con archivos temporales y operaciones del sistema.
import tempfile  # Herramientas para crear archivos y directorios temporales.
import os  # Herramientas relacionadas con el sistema operativo, como la manipulación de rutas.

# subprocess permite interactuar con comandos y programas externos desde Python.
import subprocess

# io proporciona las herramientas básicas de Python para trabajar con flujos de datos (streams).
import io  
from io import BytesIO  # Clase para trabajar con flujos de datos en memoria como si fueran archivos.


<a id = "definición"></a>
## 2. Definición de Funciones para el Procesamiento de Audio
Se presentan a continuación las funciones que se han definido para llevar a cabo diversas tareas relacionadas con el análisis y procesamiento del audio.

### 2.1. Función para reducir el ruido de un audio dado
La función utiliza la biblioteca [**noisereduce**](https://pypi.org/project/noisereduce/) para reducir el ruido en una señal de audio.
Luego, la señal de audio filtrada se convierte al formato int16 antes de devolverse.

>**Parámetros:**
* **canal_de_audio (np.array):** Un arreglo numpy que representa la señal de audio a filtrar.
* **fs (int):** La tasa de muestreo de la señal de audio.

>**Retorna:**
* **np.array:** La señal de audio filtrada en formato int16.

In [ ]:
def reduccion_de_ruido(canal_de_audio, fs):
    # Aplicar la reducción de ruido utilizando noisereduce
    audio_filtrado_float = nr.reduce_noise(y=canal_de_audio, sr=fs)
    # Convertir el audio filtrado a formato int16
    return np.int16(audio_filtrado_float)

### 2.2. Elimina los segmentos de silencio de una señal de audio dada

Esta función primero guarda el audio en un archivo temporal en formato WAV.
Luego, utiliza la función split_on_silence de la biblioteca [**pydub**](https://pypi.org/project/pydub/) para detectar y eliminar silencios.
Finalmente, se elimina el archivo temporal y se devuelve la señal de audio sin silencios.

>**Parámetros:**
* **audio (np.array):** Un arreglo numpy que representa la señal de audio.
* **sr (int):** La tasa de muestreo de la señal de audio.
* **longitud_minima_de_silencio (int, opcional):** Duración mínima (en ms) para considerar un segmento como silencio. Por defecto es 100 ms.
* **umbral_de_silencio (int, opcional):** Umbral en dBFS por debajo del cual se considera silencio. Por defecto es -60 dBFS.

>**Retorna:**
* **np.array:** La señal de audio sin los segmentos de silencio.

In [ ]:
def eliminar_silencios(audio, sr, longitud_minima_de_silencio=100, umbral_de_silencio=-60):
    # Crear un archivo temporal para guardar el audio como WAV
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as archivo_temporal_silencios:
        sf.write(archivo_temporal_silencios.name, audio, sr, format='wav')
        # Cargar el archivo temporal como un objeto AudioSegment
        representacion_del_audio_silencios = AudioSegment.from_wav(archivo_temporal_silencios.name)
    
    # Eliminar silencios del audio usando la función split_on_silence
    representaciones_de_audio_sin_silencios = split_on_silence(representacion_del_audio_silencios, min_silence_len=longitud_minima_de_silencio, silence_thresh=umbral_de_silencio)
    audio_sin_silencios = np.concatenate([np.array(representacion_de_audio_sin_silencios.get_array_of_samples()) / 32767 for representacion_de_audio_sin_silencios in representaciones_de_audio_sin_silencios])
    #32767 -> Este valor es debido a la conversion de audio digital en int16 a punto flotante en el rango de -1 a 1 
    
    # Cerrar y eliminar el archivo temporal
    archivo_temporal_silencios.close()
    os.unlink(archivo_temporal_silencios.name)
    
    return audio_sin_silencios

### 3.3.  Normaliza una señal de audio
La función normaliza la señal de audio dividiendo cada muestra por el valor máximo absoluto de la señal.
Esto asegura que la señal resultante esté en el rango de [-1, 1].

>**Parámetros:**
* **audio (np.array):** Un arreglo numpy que representa la señal de audio a normalizar.

>**Retorna:**
* **np.array:** La señal de audio normalizada.

In [ ]:
# Función para normalizar el audio
def normalizar_audio(audio):
    return audio / np.max(np.abs(audio))

### 3.4. Guarda una señal de audio en un objeto BytesIO en formato WAV
Esta función toma una señal de audio y su tasa de muestreo, y la guarda 
en un objeto BytesIO en formato WAV. Esto permite trabajar con el audio como 
si estuviera en un archivo real, pero manteniendo todo en memoria.

>**Parámetros:**
* **audio (np.array):** Un arreglo numpy que representa la señal de audio a guardar.
* **sr (int):** La tasa de muestreo de la señal de audio.

>**Retorna:**
* **BytesIO:** Un objeto BytesIO que contiene la señal de audio en formato WAV.

In [ ]:
# Función para guardar el audio en un objeto BytesIO en formato WAV
def guardar_wav_en_la_memoria(audio, sr):
    buffer = BytesIO()
    sf.write(buffer, audio, sr, format='WAV')
    buffer.seek(0)
    return buffer

### 3.5. Convierte un objeto BytesIO con un archivo WAV a formato MP3 utilizando FFmpeg
Esta función toma un objeto BytesIO que contiene una señal de audio en formato WAV 
y lo convierte a un archivo MP3. El archivo resultante se guarda en el sistema de archivos 
en la ruta especificada.

> **Nota:** Es necesario tener instalado FFmpeg y que este esté disponible en el PATH del sistema.

>**Parámetros:**
* **buffer_de_entrada (BytesIO):** Objeto BytesIO que contiene la señal de audio en formato WAV.
* **ruta_de_salida_normalizada (str):** Ruta donde se guardará el archivo MP3 resultante.

>**Retorna:**
* Esta función no devuelve ningún valor.

In [ ]:
# Función para convertir un objeto BytesIO con un archivo WAV a MP3 utilizando FFmpeg
def convertir_wav_a_mp3(buffer_de_entrada, ruta_de_salida_normalizada):
    comando = ['ffmpeg', '-i', '-', '-codec:a', 'libmp3lame', '-qscale:a', '2', ruta_de_salida_normalizada]
    proceso = subprocess.Popen(comando, stdin=subprocess.PIPE, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    proceso.communicate(input=buffer_de_entrada.read())

### 3.6. Aplica un filtro de preénfasis a una señal dada
El preénfasis es una técnica que acentúa o amplifica las altas frecuencias de una señal 
para equilibrar su espectro. Es comúnmente utilizado en el procesamiento de voz para 
mejorar la calidad de operaciones posteriores.

La formula del filtro de preénfasis es:
$y[n] = x[n] - \alpha \times x [n-1]$

Donde:
- $y[n])$ es la señal de salida.
- $x[n]$ es la señal de entrada.
- $\alpha$ es el coeficiente de preénfasis.

>**Parámetros:**
* **senial (np.array):** Un arreglo numpy que representa la señal a la que se le aplicará el preénfasis.
* **alpha (float, opcional):** El coeficiente de preénfasis. Por defecto es 0.97.

>**Retorna:**
* **np.array:** La señal con preénfasis aplicado.

In [ ]:
def filtro_preenfasis(senial, alpha=0.97):
    return np.append(senial[0], senial[1:] - alpha * senial[:-1])

### 3.7.  Segmenta y procesa un archivo de audio 
La función carga un archivo de audio y lo segmenta en frames utilizando una ventana deslizante. 
Además, aplica un filtro de preénfasis y una ventana de Hamming a cada frame.

>**Parámetros:**
* **nombre_del_archivo_de_audio_a_filtrar_Segmentar_ventanear (str):** Ruta del archivo de audio a procesar.
* **duracion_del_frame (int, opcional):** Duración de cada frame en milisegundos. Por defecto es 20 ms.
* **minima_duracion_del_audio_en_seg (int, opcional):** Duración mínima del audio en segundos para procesar. Por defecto es 2 segundos.
* **frames_totales (int, opcional):** Número máximo de frames a considerar. Por defecto es 100.

>**Retorna:**
* **tuple:** Una tupla con dos np.arrays. El primero contiene frames con el filtro de preénfasis y ventana de Hamming aplicados. El segundo contiene frames originales sin ningún procesamiento.

In [ ]:
def segmentacion_del_audio(nombre_del_archivo_de_audio_a_filtrar_Segmentar_ventanear, 
                           duracion_del_frame=20, 
                           minima_duracion_del_audio_en_seg=2, 
                           frames_totales=100):
    
    # Cargar el archivo de audio
    senial_original_a_filtrar_segmentar_ventanear, sr_a_filtrar_segmentar_ventanear = librosa.load(nombre_del_archivo_de_audio_a_filtrar_Segmentar_ventanear, sr=None)
   
    # Verificar si el audio tiene una duración mínima de 2 segundos
    if len(senial_original_a_filtrar_segmentar_ventanear) < minima_duracion_del_audio_en_seg * sr_a_filtrar_segmentar_ventanear:
        return None
    
    # Aplicar el filtro de preénfasis
    senial_filtrada_con_preenfasis = filtro_preenfasis(senial_original_a_filtrar_segmentar_ventanear)
    
    # Calcular la longitud de la ventana y el paso (hop) en muestras
    longitud_de_frame = int(sr_a_filtrar_segmentar_ventanear * duracion_del_frame / 1000)
    solapamiento = longitud_de_frame // 2

    # Aplicar la ventana deslizante para extraer los frames
    frames_con_preenfasis = librosa.util.frame(senial_filtrada_con_preenfasis, frame_length=longitud_de_frame, hop_length=solapamiento)
    frames_sin_preenfasis = librosa.util.frame(senial_original_a_filtrar_segmentar_ventanear, frame_length=longitud_de_frame, hop_length=solapamiento)
    
    # Limitar la cantidad de frames a frames_totales
    frames_con_preenfasis = frames_con_preenfasis[:, :frames_totales]
    frames_sin_preenfasis = frames_sin_preenfasis[:, :frames_totales]

    # Aplicar la ventana de Hamming a cada frame
    ventana_de_hamming = np.hamming(longitud_de_frame)
    frames_con_preenfasis = frames_con_preenfasis * ventana_de_hamming[:, np.newaxis]

    return frames_con_preenfasis, frames_sin_preenfasis


### 3.8. Guarda los frames en un archivo utilizando el formato numpy
Esta función toma un conjunto de frames (representados como un arreglo numpy) y los guarda 
en un archivo en el formato específico de numpy (.npy).

>**Parámetros:**
* **frames (np.array):** Arreglo numpy que contiene los frames a guardar.
* **ruta_de_archivo_de_salida_segmentado (str):** Ruta donde se guardará el archivo con los frames.

>**Retorna:**
* Esta función no devuelve ningún valor.

In [ ]:
def guardar_frames_en_un_archivo(frames, ruta_de_archivo_de_salida_segmentado):
    np.save(ruta_de_archivo_de_salida_segmentado, frames)

### 3.9. Carga los frames de un archivo guardado en el formato numpy
Esta función carga un conjunto de frames (representados como un arreglo numpy) desde un archivo 
que ha sido guardado en el formato específico de numpy (.npy).

>**Parámetros:**
* **archivo_de_entrada (str):** Ruta del archivo que contiene los frames guardados en formato numpy.

>**Retorna:**
* **np.array:** Arreglo numpy que contiene los frames cargados del archivo.

In [ ]:
def load_frames_from_file(archivo_de_entrada):
    return np.load(archivo_de_entrada)

<a id = "proceso_ruido"></a>
## 3. Proceso de Reducción de Ruido
En esta sección, se lleva a cabo el proceso de reducción de ruido en un archivo de audio específico. Se emplea una técnica que busca minimizar el ruido presente para mejorar la claridad del audio. Este proceso se encarga de cargar un archivo de audio en formato MP3, separar sus canales (si es estéreo), cambiar la tasa de muestreo, aplicar un proceso de reducción de ruido y, finalmente, guardar la versión filtrada.

In [ ]:
# Nombre del archivo de audio original
nombre_del_archivo_de_audio_a_quitarle_el_ruido = "audio_prueba.mp3"

# 1. Cargar el archivo de audio
representacion_del_audio_ruido = AudioSegment.from_mp3(nombre_del_archivo_de_audio_a_quitarle_el_ruido)
muestras_ruido = np.array(representacion_del_audio_ruido.get_array_of_samples())
fs_original = representacion_del_audio_ruido.frame_rate

# 2. Definir la tasa de muestreo objetivo
fs_objetivo = 32000

# 3. Separar los canales si el audio es estéreo
if representacion_del_audio_ruido.channels == 2:
    muestras_del_canal_izquierdo = muestras_ruido[::2]
    muestras_del_canal_derecho = muestras_ruido[1::2]
else:
    muestras_del_canal_izquierdo = muestras_ruido

# 4. Cambiar la tasa de muestreo a la tasa objetivo para cada canal
muestras_del_canal_izquierdo = librosa.resample(muestras_del_canal_izquierdo.astype(np.float32), orig_sr=fs_original, target_sr=fs_objetivo)
if representacion_del_audio_ruido.channels == 2:
    muestras_del_canal_derecho = librosa.resample(muestras_del_canal_derecho.astype(np.float32), orig_sr=fs_original, target_sr=fs_objetivo)

# 5. Aplicar la reducción de ruido a cada canal de manera independiente
muestras_filtradas_del_canal_izquierdo = reduccion_de_ruido(muestras_del_canal_izquierdo, fs_objetivo)
if representacion_del_audio_ruido.channels == 2:
    muestras_filtradas_del_canal_derecho = reduccion_de_ruido(muestras_del_canal_derecho, fs_objetivo)

# 6. Si el audio es estéreo, combinar los canales filtrados 
if representacion_del_audio_ruido.channels == 2:
    muestras_filtradas = np.empty(muestras_filtradas_del_canal_izquierdo.size + muestras_filtradas_del_canal_derecho.size, dtype=np.int16)
    muestras_filtradas[::2] = muestras_filtradas_del_canal_izquierdo
    muestras_filtradas[1::2] = muestras_filtradas_del_canal_derecho
else:
    muestras_filtradas = muestras_filtradas_del_canal_izquierdo

# 7. Crear una representación de audio filtrado
representacion_del_audio_filtrado = AudioSegment(
    muestras_filtradas.tobytes(),
    frame_rate=fs_objetivo,
    sample_width=muestras_filtradas.dtype.itemsize,
    channels=representacion_del_audio_ruido.channels
)

# 8. Definir la ruta de archivo de salida y guardar el audio filtrado en formato MP3
ruta_de_archivo_de_salida_ruido = os.path.splitext(nombre_del_archivo_de_audio_a_quitarle_el_ruido)[0] + '_con_reduccion_de_ruido.mp3'
representacion_del_audio_filtrado.export(ruta_de_archivo_de_salida_ruido, format="mp3")

<a id = "proceso_silencios"></a>
## 4. Proceso de Eliminación de Silencios
El audio puede tener silencios o pausas que no aportan información. En esta sección, se identifican y eliminan dichos silencios. Este proceso se encarga de cargar un archivo de audio previamente procesado (en el que se ha reducido el ruido), eliminar los silencios largos que pueda tener, y finalmente guardar el audio procesado en un nuevo archivo MP3.

In [ ]:
# 1. Definir la ruta al archivo de entrada
nombre_del_archivo_de_audio_a_eliminar_silencios = ruta_de_archivo_de_salida_ruido

# 2. Cargar el archivo de audio
audio_silencios, sr_silencios = librosa.load(nombre_del_archivo_de_audio_a_eliminar_silencios, sr=None)

# 3. Eliminar los silencios del audio
audio_sin_silencios = eliminar_silencios(audio_silencios, sr_silencios)

# 4. Definir la ruta del archivo de salida
ruta_de_salida_de_archivo_silencios = os.path.splitext(nombre_del_archivo_de_audio_a_quitarle_el_ruido)[0] + '_sin_silencios_largos.mp3'

# 5. Guardar el audio procesado en un archivo MP3
with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as archivo_temporal_silencios:
    # Convertir el audio a formato WAV y guardar en un archivo temporal
    sf.write(archivo_temporal_silencios.name, audio_sin_silencios, sr_silencios, format='wav')
    # Convertir el archivo WAV temporal a MP3 y guardar en la ruta definida
    representacion_del_audio_silencios = AudioSegment.from_wav(archivo_temporal_silencios.name)
    representacion_del_audio_silencios.export(ruta_de_salida_de_archivo_silencios, format='mp3')
    # Cerrar y eliminar el archivo temporal
    archivo_temporal_silencios.close()
    os.unlink(archivo_temporal_silencios.name)

<a id = "proceso_normalización"></a>
## 5. Proceso de Normalización del Audio
Para asegurar que el audio tenga un volumen uniforme, se aplica un proceso de normalización que ajusta la amplitud del audio. Este proceso toma un archivo de audio en el que previamente se han eliminado los silencios largos, lo normaliza (ajusta el volumen para que su amplitud máxima sea 1) y luego guarda el resultado en un nuevo archivo MP3.

In [ ]:
# 1. Cargar el archivo de audio previamente procesado (silencios eliminados)
nombre_del_archivo_de_audio_a_normalizar = ruta_de_salida_de_archivo_silencios
audio_a_normalizar, sr_a_normalizar = librosa.load(nombre_del_archivo_de_audio_a_normalizar, sr=None)

# 2. Aplicar la normalización al audio
audio_normalizado = normalizar_audio(audio_a_normalizar)

# 3. Guardar el audio normalizado en un objeto temporal (BytesIO) en formato WAV
buffer_wav_de_salida = guardar_wav_en_la_memoria(audio_normalizado, sr_a_normalizar)

# 4. Definir la ruta de salida para el audio normalizado y convertirlo a formato MP3
salida_del_audio_normalizado = os.path.splitext(nombre_del_archivo_de_audio_a_quitarle_el_ruido)[0] + '_normalizado.mp3'
convertir_wav_a_mp3(buffer_wav_de_salida, salida_del_audio_normalizado)

<a id = "proceso_segmentación"></a>
## 6. Proceso de Preenfasis, Segmentación y Ventaneo
Se aplican técnicas adicionales al audio, como el preénfasis, para mejorar ciertas características del sonido. Además, se segmenta el audio y se aplica un proceso de ventaneo. Este proceso toma un archivo de audio que ha sido previamente normalizado, lo segmenta en frames utilizando la función segmentacion_del_audio (que también aplica un filtro de preénfasis), y finalmente guarda estos frames en archivos .npy utilizando el formato específico de NumPy.

In [ ]:
# 1. Definir el nombre del archivo de audio a procesar
nombre_del_archivo_de_audio_a_filtrar_Segmentar_ventanear = salida_del_audio_normalizado

# 2. Segmentar el audio en frames
[frames_con_preenfasis, frames_sin_preenfasis] = segmentacion_del_audio(nombre_del_archivo_de_audio_a_filtrar_Segmentar_ventanear)

# 3. Guardar los frames con preénfasis en un archivo numpy
ruta_de_archivo_de_salida_segmentado_con_preenfasis = os.path.splitext(nombre_del_archivo_de_audio_a_quitarle_el_ruido)[0] + '_frames_con_preenfasis.npy'
guardar_frames_en_un_archivo(frames_con_preenfasis, ruta_de_archivo_de_salida_segmentado_con_preenfasis)

# 4. Guardar los frames sin preénfasis en un archivo numpy
ruta_de_archivo_de_salida_segmentado_sin_preenfasis = os.path.splitext(nombre_del_archivo_de_audio_a_quitarle_el_ruido)[0] + '_frames_sin_preenfasis.npy'
guardar_frames_en_un_archivo(frames_sin_preenfasis, ruta_de_archivo_de_salida_segmentado_sin_preenfasis)

<a id = "archivos"></a>
## 7. Archivos Generados

Durante el proceso, se han generado varios archivos de audio que representan cada paso del procesamiento. Estos archivos se pueden encontrar en la carpeta [recursos](https://github.com/JazminPS/Gender-and-Age-Recognition-System-from-Speech/tree/main/recursos) y son:

1. `audio_prueba_con_reduccion_de_ruido.mp3`: Audio con ruido reducido.
2. `audio_prueba_sin_silencios_largos.mp3`: Audio sin silencios prolongados.
3. `audio_prueba_normalizado.mp3`: Audio normalizado.

Además, se han guardado segmentos de audio en formato `.npy` que representan la señal después del preénfasis, segmentación, y ventaneo:

1. `audio_prueba_frames_con_preenfasis.npy`: Frames del audio con filtro preenfasis.
2. `audio_prueba_frames_sin_preenfasis.npy`: Frames del audio sin filtro preenfasis.

<a id = "gráficas"></a>
## 8. Gráficas

Las gráficas a continuación ilustran el proceso y resultados de la reducción de ruido, eliminación de silencios largos, normalización y segmentación del audio.

### 8.1. Señal de audio filtrada para la reducción del ruido

Se observa que se eliminaron algunas frecuencias en el espectro de la señal debido al filtrado del ruido. De igual manera se observa una señal más definida en el dominio del tiempo.

![Comparación entre la señal de audio original y la señal filtrada para la reducción del ruido](https://raw.githubusercontent.com/JazminPS/Gender-and-Age-Recognition-System-from-Speech/main/recursos/Se%C3%B1al_con_Reduccion_de_Ruido.png
)

### 8.2. Señal de audio sin silencios largos

Se observa que se redujo la duración de la señal en el dominio del tiempo debido a la eliminación de los silencios largos.

![Comparación entre la señal de audio original y la señal sin silencios largos](https://raw.githubusercontent.com/JazminPS/Gender-and-Age-Recognition-System-from-Speech/main/recursos/Señal_sin_silencios_largos.png)

### 8.3. Señal de audio normalizada

Se observa que la forma de las señales tanto en el dominio del tiempo como en el dominio de la frecuencia es la misma, no se modificó, la única diferencia se encuentra en las amplitudes de cada señal, ya que la normalizada varía entre [-1 y 1].

![Comparación entre la señal de audio original y la señal normalizada](https://raw.githubusercontent.com/JazminPS/Gender-and-Age-Recognition-System-from-Speech/main/recursos/Señal_normalizada.png)

### 8.4. Señal filtrada con un filtro preénfasis

Como se muestra en la gráfica en el espectro de la señal se equilibran las frecuencias altas y bajas

![Comparación entre la señal de audio original y la señal con filtro preénfasis](https://raw.githubusercontent.com/JazminPS/Gender-and-Age-Recognition-System-from-Speech/main/recursos/Señal_con_filtro_preenfasis.png)

### 8.5. Segmentación y ventaneo

En la gráfica se muestra la ventana de Hamming utilizada junto con la comparación entre un frame perteneciente a la señal antes y después de aplicarle dicha ventana.
Se aprecia que al aplicarle la ventana al segmento se suavizaron los bordes lo cual justamente es el objetivo del ventaneo.


![Segmentacion y ventaneo](https://raw.githubusercontent.com/JazminPS/Gender-and-Age-Recognition-System-from-Speech/main/recursos/Segmentacion_y_ventaneo.png)